In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

In [3]:
import cv2
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [4]:
from dataset_spliter import SplitByPatient
from metrics import *#F1Weighted, MCC
from losses import *

In [5]:
import pretrainedmodels

In [6]:
path = Path('/data/Datasets/WhiteBloodCancer/train/')
test_path = Path('/data/Datasets/WhiteBloodCancer/test/')

In [7]:
np.random.seed(42)

In [8]:
fnames = get_image_files(path, recurse=True)
fnames[:5]

[PosixPath('/data/Datasets/WhiteBloodCancer/train/fold_1/hem/UID_H10_43_1_hem.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/train/fold_1/hem/UID_H22_31_15_hem.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/train/fold_1/hem/UID_H14_9_11_hem.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/train/fold_1/hem/UID_H14_28_6_hem.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/train/fold_1/hem/UID_H10_189_1_hem.bmp')]

In [9]:
len(fnames)

12528

In [10]:
train_files_regex = re.compile(r'(fold_0|fold_1|fold_2)')

fnames = [fn for fn in fnames if train_files_regex.search(str(fn)) is not None]
len(fnames)

10661

In [11]:
hem_regex = re.compile(r'UID_(H[0-9]+)_', re.IGNORECASE)
all_regex = re.compile(r'UID_([0-9]+)_', re.IGNORECASE)

In [12]:
hem_patient_ids = list(set([hem_regex.search(str(fn)).group(1)
                            for fn in fnames if hem_regex.search(str(fn)) is not None]))
all_patint_ids = list(set([all_regex.search(str(fn)).group(1)
                           for fn in fnames if all_regex.search(str(fn)) is not None]))

hem_patients = dict((k,[]) for k in hem_patient_ids)
all_patints = dict((k,[]) for k in all_patint_ids)

[all_patints[key].append(fn) for key in all_patints.keys() for fn in fnames if 'UID_{0}_'.format(key) in str(fn)]
[hem_patients[key].append(fn) for key in hem_patients.keys() for fn in fnames if 'UID_{0}_'.format(key) in str(fn)]
print()

## Split data into train val 

In [13]:
split_handler = SplitByPatient(hem_patients, all_patints)

In [14]:
folds = split_handler.split_by_folds(10)

In [15]:
pat = re.compile(r'^.*(hem|all).bmp$')

def get_label(fn):
    return pat.search(str(fn)).group(1)

#### Data augmentation

In [16]:
xtra_tfms=[cutout(n_holes=5, length=0.2)]#squish(scale=0.66), 
tfms = get_transforms(do_flip=True, 
                      flip_vert=True, 
                      #max_rotate=90,  
                      max_lighting=0., 
                      max_zoom=1.0, 
                      max_warp=0.0,
                      #p_affine=0.75,
                      #p_lighting=0.75,  
                      #xtra_tfms=xtra_tfms,
                     )

#### Create dataset 

In [17]:
import pandas as pd

file = "/server/born_pix/EPA_DATASETS/WhiteBloodCancer/VAL_ISBI_labelfile_Source_reference_prediction.csv"
dataset = pd.read_csv(file, delimiter=';')
gt_labels = np.array(dataset.loc[:, 'labels'])

In [18]:
def get_data(bs, size, train, val):
    
    train_il = ImageList(train) #optinal scale up classes 
    valid_il = ImageList(val)
    item_list = ItemLists(path, train_il, valid_il)
    lls = item_list.label_from_func(get_label).add_test_folder('../test')
    
    data  = ImageDataBunch.create_from_ll(lls, size=size, bs=bs, 
                                      ds_tfms=tfms)
    data = data.normalize()
    #data = data.normalize((channel_mean, channel_std))
    return data

In [19]:
def get_cadene_model(pretrained=True, model_name='se_resnext50_32x4d'):
    if pretrained:
        arch = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    else:
        arch = pretrainedmodels.__dict__[model_name](num_classes=1000)
    return arch

In [ ]:
result = {}
y_pred_list = {}
folds_nr = 10
folds = split_handler.split_by_folds(folds_nr)

for i in tqdm(range(folds_nr)):
    print(result)
    
    val_files = folds[i]
    train_files = []
    for sub in range(folds_nr):
        if sub != i:
            train_files.extend(folds[sub])
            
    size = 256
    bs = 96

    data = get_data(bs, size, train_files, val_files)
    
    
    gc.collect()
        
    experiment_name = "baseline_se_resnext50_32x4d_fold_{}".format(i)
    learn = create_cnn(data, get_cadene_model, 
                    #cut=-2,
                       metrics=[error_rate, F1Weighted(), MCC()], #  
                       #loss_func=FocalLoss(num_classes=1),
                       #ps=0.75,
                       #wd=0.1,
                       loss_func = LabelSmoothingCrossEntropy(),
                       callback_fns=[partial(SaveModelCallback, name='stage1-{}-{}'.format(experiment_name, size))],

                  )#

    for size, bs in [[256, 64], [384, 32]]:
        learn.data = get_data(bs, size, train_files, val_files)
        learn.freeze()
            
        lr = 1e-2
        learn.fit_one_cycle(5, lr)

        learn.unfreeze()
        learn.fit_one_cycle(10, slice(1e-5,lr/5))
            
        y_pred, y_test_tta = learn.TTA(ds_type=DatasetType.Test, scale=1.15)#, beta=0.4, scale=1.3
        y_pred = to_np(y_pred)
            
        submission = [0 for i in range(y_pred.shape[0])]
        for fn, y in zip(learn.data.test_dl.items, np.argmax(y_pred[:, [1,0]], axis=1)):
            index = int(fn.name.replace(".bmp", '')) - 1
            submission[index] = y

        score = f1_score(gt_labels, submission, average='weighted')
        result["{}-{}".format(i, size)] = score  
        y_pred_list["{}-{}".format(i, size)] = y_pred

 20%|██        | 2/10 [4:17:14<17:08:58, 7717.35s/it]

{'0-256': 0.8353715839488581, '0-384': 0.8440364197373512, '1-256': 0.7297822727035388, '1-384': 0.781689018119115}


/home/c.marzahl@de.eu.local/ProgProjekte/fastai_v1/fastai/fastai/vision/learner.py:93: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")


epoch,train_loss,valid_loss,error_rate,f1_weighted,mcc,time
0,0.539225,0.495423,0.140663,0.859195,0.712025,02:07
1,0.465874,0.455624,0.102668,0.896171,0.791745,02:06
2,0.428641,0.434700,0.085691,0.914152,0.824495,02:06
3,0.403082,0.425297,0.075990,0.924080,0.845068,02:05
4,0.390907,0.421391,0.073565,0.926461,0.849770,02:06


Better model found at epoch 0 with val_loss value: 0.495422899723053.
Better model found at epoch 1 with val_loss value: 0.4556242823600769.
Better model found at epoch 2 with val_loss value: 0.4347001910209656.
Better model found at epoch 3 with val_loss value: 0.42529672384262085.
Better model found at epoch 4 with val_loss value: 0.42139115929603577.


epoch,train_loss,valid_loss,error_rate,f1_weighted,mcc,time
0,0.387744,0.423701,0.075990,0.924128,0.845443,02:38
1,0.388092,0.414977,0.071140,0.928910,0.854860,02:38
2,0.386544,0.414766,0.068715,0.931399,0.860334,02:42
3,0.380462,0.412334,0.069523,0.930556,0.858371,02:41
4,0.374713,0.417322,0.075990,0.924186,0.846069,02:39
5,0.365039,0.414847,0.063864,0.936215,0.869966,02:40
6,0.364871,0.410476,0.063056,0.937067,0.872070,02:40
7,0.359901,0.408252,0.063056,0.937113,0.872730,02:40
8,0.358268,0.416680,0.069523,0.930675,0.859775,02:38


Better model found at epoch 0 with val_loss value: 0.42370131611824036.
Better model found at epoch 1 with val_loss value: 0.41497674584388733.
Better model found at epoch 2 with val_loss value: 0.41476550698280334.
Better model found at epoch 3 with val_loss value: 0.4123342037200928.
Better model found at epoch 6 with val_loss value: 0.41047558188438416.
Better model found at epoch 7 with val_loss value: 0.4082520008087158.


In [ ]:
result

In [ ]:
[y_pred_list[y][:, 0] for y in y_pred_list if "-384" in y]

In [ ]:
[y_pred_list[y][:, 1] for y in y_pred_list if "-384" in y]

In [ ]:
submission = [0 for i in range(1867)]
for i in range(10):
    ALL = [y_pred_list[y][:, 0] for y in y_pred_list if "-384" in y][i]
    normal = [y_pred_list[y][:, 1] for y in y_pred_list if "-384" in y][i]


    for fn, a, normal in zip(learn.data.test_dl.items, ALL, normal):
        index = int(fn.name.replace(".bmp", '')) - 1
        submission[index] += 1 if a > normal else 0
        
for i in range(10):
    print("{} {}".format(i, f1_score(gt_labels, (np.array(submission) > i).astype(np.int), average='weighted')))